Fonctions utiles intervenant à plusieurs endroits dans nos projets

In [2]:
# Pour chaque paire, on va récupérer les arcs dont ils sont l'origine (calcul du demi-degré extérieur des sommets du graphe)
function degreExt(kep)
    origine = Array{Array{Int64,1},1}()
    for v in 1:nv(kep)
        # On recupère les indices des arcs dont l'origine est v
        origin_v = Array{Int64,1}()
        for (ind,arc) in enumerate(edges(kep))
            if src(arc) == v
                push!(origin_v,ind)
            end
        end
        push!(origine,origin_v)
    end
    return origine
end

# Pour chaque paire, on va récupérer les arcs dont ils sont la destination (calcul du demi-degré intérieur des sommets)
function degreInt(kep)
    dest = Array{Array{Int64,1},1}()
    for v in 1:nv(kep)
        # On recupère les indices des arcs dont l'origine est v
        dest_v = Array{Int64,1}()
        for (ind,arc) in enumerate(edges(kep))
            if dst(arc) == v
                push!(dest_v,ind)
            end
        end
        push!(dest,dest_v)
    end
    return dest
end

degreInt (generic function with 1 method)

In [3]:
# Cette fonction sert a determiner dans un graphe tous les cycles de longueur inferieure ou egale a L
# on peut aussi utiliser "filter(x->length(x)<=L,simplecycles_hawick_james(pool))" mais 
# procedure plus lente
function get_cycles(pool, L)
    paires = 1:nv(pool)
    cycles = Array{Array{Int,1},1}()
    ens_arcs = collect(edges(pool))
    function rec_enum(p,c,L)
        for arc in origine[p]
            q = dst(ens_arcs[arc])
            if  q == c[1]
                push!(cycles,c)
            elseif q <= c[1]
                continue
            elseif q in c
                continue
            elseif size(c,1)<=L-1
                cc = copy(c)
                push!(cc,q)
                rec_enum(q,cc,L)
            end
        end
    end
    
    for p in paires
        if(!isempty(origine[p]))
            c = [p]
            rec_enum(p,copy(c),L)
        end
    end
    
    # renvoie les cycles et les sommets qui le composent
    return cycles
end

get_cycles (generic function with 1 method)

In [4]:
# Utilise la sortie de get_cycles (sommets qui composent les cycles)
# et la transforme en une sortie qui considère les numéros des arcs composant le cycle
function get_cycles_arcs(cycles_paires)
    ens_arcs = collect(edges(pool))
    cycles_arcs = Array{Array{Int,1},1}()
    for cycles in cycles_paires
        c = []
        ind = 1
        org = cycles[1]
        while(ind < size(cycles,1))
            for arc in origine[org]
                if ind < size(cycles,1)
                    if dst(ens_arcs[arc]) == cycles[ind+1]
                        push!(c,arc)
                        org = cycles[ind+1]
                        ind = ind+1
                        break
                    end
                end
            end
        end
        if ind == size(cycles,1)
            org1 = cycles[ind]
            for arc in origine[org1]
                if dst(ens_arcs[arc])==cycles[1]
                    push!(c,arc)
                end
            end
        end
        push!(cycles_arcs,c)
    end
    return cycles_arcs
end

get_cycles_arcs (generic function with 1 method)

In [5]:
# Utilisation des fonctions degreExt et degreInt
origine = degreExt(pool); # pour chaque sommet, numero des arcs dont ils sont a l'origine
dest = degreInt(pool); # pour chaque sommet numero des arcs dont ils sont la destination

LoadError: UndefVarError: pool not defined